### Hotmart Technical Case: Analysis about Sales Recovery
Sales recovery refers to the strategy or tools that help complete 
sales for those who, for some reason, did not complete them. 
It is a resource that should be applied to any online business. This is 
because it allows you to rescue people who did not complete the 
purchase for different reasons: lack of limit on the card, balance in 
the account, objections to the purchase, among others.
One of Hotmart tools recovers sales when the customer does not 
have a sufficient limit on their credit card.
When an attempt to purchase in installments is denied due to 
insufficient balance on the part of the Buyer, the tool transforms the 
transaction into a recurring one to prevent the sale from being lost.
Thus, the buyer will receive monthly charges from the Hotmart 
Payment System until the total purchase amount is paid.

- **Problem**: The main problem is sales that are not complete, it can be for sales cancelled and refund. 
- **Goals**: Analysis of the data to recognize stands that will help to identify the main reasons that the people are not completely yours shopping. Besides, find other ways to improve the sales rate completed, in addition to the recovery tool shown in case text.
- **Database**: There are three tables (producers, products and sales)


In **this notebook** I intend to perform an initial data exploration and data cleaning, necessary to explore the data and to work SQL after that.

In [44]:
import pandas as pd
import sqlalchemy

### 1. Data Validation and Clean

#### 1.1 Loading data

In [45]:
df_producers = pd.read_csv(r"..\..\data\Producers.csv", sep=",", index_col=False)
df_producers = df_producers.reset_index(drop=True)
df_producers.head(5)

,producer_id,registry_date,country
0,1,2018-02-01,BRASIL
1,2,2018-01-04,COLOMBIA
2,3,2020-01-02,BRASIL
3,4,2021-01-02,COLOMBIA
4,5,2019-02-11,BRASIL


In [46]:
# Checking type of the data
df_producers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15 entries, 0 to 14
Data columns (total 3 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   producer_id    15 non-null     int64 
 1   registry_date  15 non-null     object
 2   country        15 non-null     object
dtypes: int64(1), object(2)
memory usage: 492.0+ bytes


In [47]:
df_products = pd.read_csv(r"..\..\data\Products.csv", sep=",", index_col=False)
df_products.head(5)

,product_id,producer_id,registry_date,recovery_active,member_area_active,deletion_date,niche,base_price,type
0,218834610,1,2020-03-30,1,1,NaN,Marketing,780,Curso
1,253458328,4,2021-05-24,0,0,NaN,Educação Infantil,560,Assinatura
2,466262361,6,2020-11-13,1,1,NaN,Empreendedorismo,800,Curso
3,613311395,2,2018-02-28,0,0,NaN,Financas,680,Curso
4,528943857,2,2018-09-15,1,0,NaN,Financas,500,Curso


In [48]:
# Checking type of the data
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   product_id          75 non-null     int64 
 1   producer_id         75 non-null     int64 
 2   registry_date       75 non-null     object
 3   recovery_active     75 non-null     int64 
 4   member_area_active  75 non-null     int64 
 5   deletion_date       12 non-null     object
 6   niche               75 non-null     object
 7   base_price          75 non-null     object
 8   type                75 non-null     object
dtypes: int64(4), object(5)
memory usage: 5.4+ KB


In [49]:
# Exploratiory column "base_price", because would most be with numeric format

df_products["base_price"].unique

<bound method Series.unique of 0          780
1          560
2          800
3          680
4          500
        ...   
70         800
71         480
72    1,520.00
73         868
74         968
Name: base_price, Length: 75, dtype: object>

In [50]:
# Adjusting column "base_price", because would most be with numeric format
# The reason was some values with milhar and decimal separator

df_products["base_price"] = (
     df_products["base_price"]
    .str.replace(",", "")
)
df_products

,product_id,producer_id,registry_date,recovery_active,member_area_active,deletion_date,niche,base_price,type
0,218834610,1,2020-03-30,1,1,NaN,Marketing,780,Curso
1,253458328,4,2021-05-24,0,0,NaN,Educação Infantil,560,Assinatura
2,466262361,6,2020-11-13,1,1,NaN,Empreendedorismo,800,Curso
3,613311395,2,2018-02-28,0,0,NaN,Financas,680,Curso
4,528943857,2,2018-09-15,1,0,NaN,Financas,500,Curso
...,...,...,...,...,...,...,...,...,...
70,241258270,6,2021-03-06,1,1,2021-04-06,Empreendedorismo,800,Assinatura
71,732676177,6,2021-03-13,1,1,NaN,Empreendedorismo,480,Curso
72,251959622,7,2021-02-06,0,0,NaN,Saúde e Alimentação,1520.00,Palestra
73,957182962,8,2021-01-28,0,1,NaN,Tecnologia e Inovação,868,Assinatura


In [51]:
# Adjusting data formats: base_price from object to float

df_products['base_price'] = df_products['base_price'].astype(float)
df_products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 0 to 74
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   product_id          75 non-null     int64  
 1   producer_id         75 non-null     int64  
 2   registry_date       75 non-null     object 
 3   recovery_active     75 non-null     int64  
 4   member_area_active  75 non-null     int64  
 5   deletion_date       12 non-null     object 
 6   niche               75 non-null     object 
 7   base_price          75 non-null     float64
 8   type                75 non-null     object 
dtypes: float64(1), int64(4), object(4)
memory usage: 5.4+ KB


In [52]:
df_sales = pd.read_csv(r"..\..\data\Sales.csv", sep=",", index_col=False)
df_sales.head(5)

,purchase_id,purchase_date,product_id,product_price,service_tax,comission_value,has_coupon,discount,refund,cancelled,chargeback
0,696733696,2022-06-21,275653577,48.00,7.2,36.00,1,0.1,0,0,0
1,957560528,2021-01-02,399974968,45.00,4.5,40.50,0,0.0,1,0,0
2,800828744,2020-07-07,489748266,"1,000.00",100.0,900.00,0,0.0,0,0,0
3,964632300,2022-11-24,545252229,210.00,21.0,168.00,1,0.1,0,0,0
4,830193347,2021-09-05,489748266,"1,000.00",100.0,900.00,0,0.0,0,0,0


In [53]:
# Checking type of the data
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10683 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   purchase_id      10683 non-null  int64  
 1   purchase_date    10683 non-null  object 
 2   product_id       10683 non-null  int64  
 3   product_price    10683 non-null  object 
 4   service_tax      10683 non-null  float64
 5   comission_value  10683 non-null  object 
 6   has_coupon       10683 non-null  int64  
 7   discount         10683 non-null  float64
 8   refund           10683 non-null  int64  
 9   cancelled        10683 non-null  int64  
 10  chargeback       10683 non-null  int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 918.2+ KB


In [54]:
# Checking column purchase_date

dt_purchase_min = df_sales['purchase_date'].min()

print(dt_purchase_min)
display(df_sales[df_sales['purchase_date'] == '1900-01-01'])

1900-01-01


,purchase_id,purchase_date,product_id,product_price,service_tax,comission_value,has_coupon,discount,refund,cancelled,chargeback
2988,475660961,1900-01-01,489748266,"1,000.00",120.0,880.00,0,0.0,0,0,1
5033,333836315,1900-01-01,399974968,45.00,5.4,35.10,1,0.1,0,0,0
5056,250855304,1900-01-01,253458328,560.00,56.0,392.00,1,0.2,0,0,1
7139,942718796,1900-01-01,585809781,830.00,83.0,747.00,0,0.0,0,0,0
7154,204871933,1900-01-01,489748266,"1,000.00",120.0,780.00,1,0.1,0,0,0
8242,896848608,1900-01-01,399974968,45.00,5.4,39.60,0,0.0,0,1,0


In [55]:
# How there are a lot of lines,it was decided remove line with value: '1900-01-01'
# We can understand que this value is null date

df_sales = df_sales[df_sales['purchase_date'] != '1900-01-01']
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10677 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   purchase_id      10677 non-null  int64  
 1   purchase_date    10677 non-null  object 
 2   product_id       10677 non-null  int64  
 3   product_price    10677 non-null  object 
 4   service_tax      10677 non-null  float64
 5   comission_value  10677 non-null  object 
 6   has_coupon       10677 non-null  int64  
 7   discount         10677 non-null  float64
 8   refund           10677 non-null  int64  
 9   cancelled        10677 non-null  int64  
 10  chargeback       10677 non-null  int64  
dtypes: float64(2), int64(6), object(3)
memory usage: 1001.0+ KB


In [56]:
# Adjusting column "product_price", because would most be with numeric format
# The reason was some values with milhar and decimal separator

df_sales["product_price"] = (
     df_sales["product_price"]
    .str.replace(",", "")
)

df_sales["comission_value"] = (
     df_sales["comission_value"]
    .str.replace(",", "")
)

df_sales

,purchase_id,purchase_date,product_id,product_price,service_tax,comission_value,has_coupon,discount,refund,cancelled,chargeback
0,696733696,2022-06-21,275653577,48.00,7.2,36.00,1,0.1,0,0,0
1,957560528,2021-01-02,399974968,45.00,4.5,40.50,0,0.0,1,0,0
2,800828744,2020-07-07,489748266,1000.00,100.0,900.00,0,0.0,0,0,0
3,964632300,2022-11-24,545252229,210.00,21.0,168.00,1,0.1,0,0,0
4,830193347,2021-09-05,489748266,1000.00,100.0,900.00,0,0.0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
10678,289956962,2020-04-04,399974968,45.00,5.4,39.60,0,0.0,0,0,0
10679,788921762,2021-01-03,489748266,1000.00,100.0,900.00,0,0.0,0,0,1
10680,345494529,2022-02-04,264261718,560.00,56.0,504.00,0,0.0,0,0,0
10681,199287415,2022-09-30,399974968,45.00,4.5,36.00,1,0.1,0,1,1


In [57]:
# Adjusting data formats: (product_price e comission_value) from object to float

df_sales['product_price'] = df_sales['product_price'].astype(float)
df_sales['comission_value'] = df_sales['comission_value'].astype(float)
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10677 entries, 0 to 10682
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   purchase_id      10677 non-null  int64  
 1   purchase_date    10677 non-null  object 
 2   product_id       10677 non-null  int64  
 3   product_price    10677 non-null  float64
 4   service_tax      10677 non-null  float64
 5   comission_value  10677 non-null  float64
 6   has_coupon       10677 non-null  int64  
 7   discount         10677 non-null  float64
 8   refund           10677 non-null  int64  
 9   cancelled        10677 non-null  int64  
 10  chargeback       10677 non-null  int64  
dtypes: float64(4), int64(6), object(1)
memory usage: 1001.0+ KB


In [58]:
# Verify consistency columns: refund e cancelled
# Because I undenstand in one sale can only happend once one of events. 
# Therefore, it must to have only type of signaling sale not completed (refund or cancelled)

df_sales_valid = df_sales.copy()
df_sales_valid['total_not_completed_sale'] = df_sales[['refund', 'cancelled']].sum(axis=1)

df_sales_valid[df_sales_valid['total_not_completed_sale'] > 1]

# There isn't inconsistency

,purchase_id,purchase_date,product_id,product_price,service_tax,comission_value,has_coupon,discount,refund,cancelled,chargeback,total_not_completed_sale


#### 1.2 Missing Values

In [59]:
df_producers.isnull().any()

producer_id      False
registry_date    False
country          False
dtype: bool

In [60]:
df_products.isnull().any()

product_id            False
producer_id           False
registry_date         False
recovery_active       False
member_area_active    False
deletion_date          True
niche                 False
base_price            False
type                  False
dtype: bool

In [61]:
# Checking table lines with column: 'deletion_date' is not null
df_products[df_products["deletion_date"].isnull() == False]

,product_id,producer_id,registry_date,recovery_active,member_area_active,deletion_date,niche,base_price,type
27,325563626,8,2019-01-11,0,0,2020-08-25,Marketing,660.0,Assinatura
33,534583965,2,2019-12-18,0,0,2021-02-18,Financas,35.0,Ebook
34,603531440,5,2019-05-30,1,0,2021-06-14,Financas,580.0,Produto Físico
50,585123782,3,2022-07-23,1,0,2022-07-23,Tecnologia e Inovação,700.0,Palestra
51,585253783,2,2019-01-04,1,1,2019-01-04,Financas,550.0,Curso
52,596809784,5,2021-01-01,0,0,2021-01-01,Empreendedorismo,48.0,Ebook
53,965497850,6,2020-10-02,1,0,2020-10-02,Saúde e Alimentação,620.0,Palestra
54,858509786,6,2021-11-02,1,1,2021-11-02,Saúde e Alimentação,590.0,Assinatura
57,352679275,4,2021-08-10,0,0,2021-08-12,Educação Infantil,392.0,Palestra
63,474268340,13,2019-01-04,1,1,2019-01-24,Tecnologia e Inovação,1984.0,Assinatura


After analysis, understand that the column **deletion_date** is not null, it is products exclued.

In [62]:
df_sales.isnull().any()

purchase_id        False
purchase_date      False
product_id         False
product_price      False
service_tax        False
comission_value    False
has_coupon         False
discount           False
refund             False
cancelled          False
chargeback         False
dtype: bool

#### 1.3 Duplicates Values

In [63]:
df_producers.duplicated(keep=False).sum()

0

In [64]:
df_products.duplicated(keep=False).sum()

0

In [65]:
df_sales.duplicated(keep=False).sum()

11

In [66]:
# Checking duplicate purchase_id in the table: sales
df_sales[df_sales.duplicated(keep=False)]

,purchase_id,purchase_date,product_id,product_price,service_tax,comission_value,has_coupon,discount,refund,cancelled,chargeback
277,939635730,2022-11-14,251959622,1520.0,152.00,1368.00,0,0.0,0,1,0
278,939635730,2022-11-14,251959622,1520.0,152.00,1368.00,0,0.0,0,1,0
279,939635730,2022-11-14,251959622,1520.0,152.00,1368.00,0,0.0,0,1,0
411,410768897,2021-11-13,489748266,1000.0,150.00,850.00,0,0.0,0,0,0
413,410768897,2021-11-13,489748266,1000.0,150.00,850.00,0,0.0,0,0,0
7899,364212656,2020-12-12,983844542,650.0,65.00,585.00,0,0.0,0,0,0
7900,364212656,2020-12-12,983844542,650.0,65.00,585.00,0,0.0,0,0,0
7924,216344463,2021-10-07,399463467,1488.0,178.56,1309.44,0,0.0,0,0,0
7926,216344463,2021-10-07,399463467,1488.0,178.56,1309.44,0,0.0,0,0,0
8026,742652675,2022-07-16,780973554,950.0,114.00,836.00,0,0.0,0,0,1


In [67]:
# Removing data dupliacates
df_sales = df_sales[df_sales.duplicated(keep=False) == False]
df_sales.duplicated(keep=False).sum()

0

#### 1.4 Outliers Analysis

In [68]:
df_products[["recovery_active", "member_area_active", "base_price"]].describe()

,recovery_active,member_area_active,base_price
count,75.000000,75.000000,75.000000
mean,0.653333,0.400000,623.560000
std,0.479113,0.493197,437.075856
min,0.000000,0.000000,30.000000
25%,0.000000,0.000000,401.000000
50%,1.000000,0.000000,590.000000
75%,1.000000,1.000000,790.000000
max,1.000000,1.000000,1984.000000


In [69]:
df_sales.iloc[:, 3:].describe()

,product_price,service_tax,comission_value,has_coupon,discount,refund,cancelled,chargeback
count,10666.000000,10666.000000,10666.000000,10666.000000,10666.000000,10666.000000,10666.000000,10666.000000
mean,511.226514,56.826059,423.142674,0.402869,0.060698,0.077349,0.113632,0.052128
std,528.354758,59.923432,441.392283,0.490498,0.085226,0.267156,0.317379,0.222296
min,30.000000,3.000000,16.500000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,48.000000,5.400000,40.500000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,390.000000,39.000000,324.800000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,950.000000,100.000000,720.000000,1.000000,0.100000,0.000000,0.000000,0.000000
max,1984.000000,297.600000,1785.600000,1.000000,0.300000,1.000000,1.000000,1.000000


It was considered columns with coerent values accordingly with bussiness understanding. That's why didn't need more analysis, like box plot charts.

#### 2. Create db with SQLAlchemy Lib

After analysis about data quality, it was decided use of SQLAlchemy lib for SQL Manipulation. That's why bellow will have the code for create db using the SQL Lite besides of SQLAlchemy lib: 

In [70]:
path_file = "../../data/"
engine = sqlalchemy.create_engine(f"sqlite:///{path_file}sales.db")

# Dataframes with table names
dataframes = [
    ("tb_producers", df_producers),
    ("tb_products", df_products),
    ("tb_sales", df_sales),
]

for table_name, df in dataframes:
    df.to_sql(name=table_name, con=engine, if_exists="replace", index=False)
    print(f"Table '{table_name}' cried with sucess.")

Table 'tb_producers' cried with sucess.
Table 'tb_products' cried with sucess.
Table 'tb_sales' cried with sucess.


In [72]:
df_producers.to_csv('../../data/producers_v2.csv', index=False)
df_products.to_csv('../../data/products_v2.csv', index=False)
df_sales.to_csv('../../data/sales_v2.csv', index=False)